In [1]:
import cv2
import face_recognition
import numpy as np
import dlib
import csv
from datetime import datetime

In [3]:
known_face_encodings = []
known_face_names = []

# Load and encode known faces (replace with your actual image paths)
image1 = face_recognition.load_image_file(r'C:\Users\Admin\Pictures\Camera Roll\WIN_20240910_15_41_11_Pro.jpg')
image2 = face_recognition.load_image_file(r'C:\Users\Admin\Pictures\Camera Roll\WIN_20240910_15_41_11_Pro.jpg')

# Encode the faces
john_face_encoding = face_recognition.face_encodings(image1)[0]
jane_face_encoding = face_recognition.face_encodings(image2)[0]

# Append the encodings and names to lists
known_face_encodings.append(john_face_encoding)
known_face_names.append("John Doe")

known_face_encodings.append(jane_face_encoding)
known_face_names.append("Jane Doe")

In [30]:


# Initialize the webcam
video_capture = cv2.VideoCapture(0)

# Load the dlib face detector and facial landmarks predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r'C:\Users\Admin\Downloads\shape_predictor_68_face_landmarks.dat')



# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Function to mark attendance in a CSV file
def mark_attendance(name):
    now = datetime.now()
    time_string = now.strftime('%Y-%m-%d %H:%M:%S')

    try:
        with open(r'C:\Users\Admin\Documents\attendance.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([name, time_string])
    except Exception as e:
        print(f"Error writing to CSV file: {e}")

# Initialize attendance state
attendance_recorded = set()

# Define constants for blink detection
EAR_THRESH = 0.2
EAR_CONSEC_FRAMES = 3
blink_counter = 0

# Start processing the video feed
while True:
    ret, frame = video_capture.read()  # Capture frame from webcam
    if not ret:
        print("Failed to grab frame")
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for face_recognition
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale for dlib

    # Find all faces and face encodings in the current frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Detect faces and landmarks
    detections = detector(gray)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Compare the face with known faces
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        
        # If a match is found, find the name of the matching face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        # Draw a rectangle around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        # Label the face with a name
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

        # Check for blink detection
        for detection in detections:
            shape = predictor(gray, detection)
            shape = np.array([(p.x, p.y) for p in shape.parts()])
            left_eye = shape[36:42]
            right_eye = shape[42:48]
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            ear = (left_ear + right_ear) / 2.0

            if ear < EAR_THRESH:
                blink_counter += 1
                if blink_counter >= EAR_CONSEC_FRAMES:
                    # Mark attendance if a blink is detected
                    if name != "Unknown" and name not in attendance_recorded:
                        mark_attendance(name)
                        attendance_recorded.add(name)
            else:
                blink_counter = 0

    # Display the result
    cv2.imshow('Video', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
video_capture.release()
cv2.destroyAllWindows()
